In [419]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

In [420]:
df = pd.read_csv('./data_with_weather.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [421]:
df.head()

,Unnamed: 0,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,day,hour,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth
0,0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,2016-03-14,17,51,40,45.5,0.29,0,0
1,1,id2129090,1,2016-03-14 14:05:39,2016-03-14 14:28:05,1,-73.975090,40.758766,-73.953201,40.765068,N,1346,2016-03-14,14,51,40,45.5,0.29,0,0
2,2,id0256505,1,2016-03-14 15:04:38,2016-03-14 15:16:13,1,-73.994484,40.745087,-73.998993,40.722710,N,695,2016-03-14,15,51,40,45.5,0.29,0,0
3,3,id3863815,2,2016-03-14 04:24:36,2016-03-14 04:37:11,3,-73.944359,40.714489,-73.910530,40.709492,N,755,2016-03-14,4,51,40,45.5,0.29,0,0
4,4,id3817493,2,2016-03-14 14:57:56,2016-03-14 15:15:26,1,-73.952881,40.766468,-73.978630,40.761921,N,1050,2016-03-14,14,51,40,45.5,0.29,0,0


In [422]:
df['distance'] = (np.abs(df['pickup_latitude']-df['dropoff_latitude'])\
+np.abs(df['pickup_longitude']-df['dropoff_longitude']))*10

In [423]:
df.head()

,Unnamed: 0,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,...,trip_duration,day,hour,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth,distance
0,0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,...,455,2016-03-14,17,51,40,45.5,0.29,0,0,0.198593
1,1,id2129090,1,2016-03-14 14:05:39,2016-03-14 14:28:05,1,-73.975090,40.758766,-73.953201,40.765068,...,1346,2016-03-14,14,51,40,45.5,0.29,0,0,0.281906
2,2,id0256505,1,2016-03-14 15:04:38,2016-03-14 15:16:13,1,-73.994484,40.745087,-73.998993,40.722710,...,695,2016-03-14,15,51,40,45.5,0.29,0,0,0.268860
3,3,id3863815,2,2016-03-14 04:24:36,2016-03-14 04:37:11,3,-73.944359,40.714489,-73.910530,40.709492,...,755,2016-03-14,4,51,40,45.5,0.29,0,0,0.388260
4,4,id3817493,2,2016-03-14 14:57:56,2016-03-14 15:15:26,1,-73.952881,40.766468,-73.978630,40.761921,...,1050,2016-03-14,14,51,40,45.5,0.29,0,0,0.302963


#### Function to grab next batch of data

In [424]:
num = 0
def next_batch(data,batch_size):
    global num
    if num*batch_size+batch_size > len(data):
        num = 0
    batch = data[num*batch_size:num*batch_size+batch_size,:]
    num = num+1
    return batch

#### Weight and bias initial function

In [425]:
# Initial Weight
def init_weight(shape):
    return tf.Variable(tf.random_normal(shape,stddev=0.1))

# Initial bias
def init_bias(shape):
    return tf.Variable(tf.constant(0.2,shape=shape))

In [426]:
gW1 = init_weight([7,14])
gb1 = init_bias([14])
gW2 = init_weight([14,4])
gb2 = init_bias([4])
gW3 = init_weight([4,1])
gb3 = init_bias([1])

def forward(z):
    fc1 = tf.sigmoid(tf.matmul(z,gW1)+gb1)
    fc2 = tf.nn.relu(tf.matmul(fc1,gW2)+gb2)
    fc3 = tf.matmul(fc2,gW3)+gb3
    return fc3

#### Create Dense Nerual Network

In [427]:
batch_size = 20
learning_rate = 0.0001

In [428]:
# Create PlaceHolder
xph = tf.placeholder(tf.float32,[None,7])
yph = tf.placeholder(tf.float32,[None,1])

In [429]:
# Error Function and optimizer
result = forward(xph)
loss = tf.reduce_sum(tf.square(yph-forward(xph)))
optimizer= tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(loss)

In [430]:
# Prepare Feed Data
whole_dataset = df[['pickup_longitude','pickup_latitude','dropoff_longitude',
             'dropoff_latitude','distance','hour','average temperature','trip_duration']].as_matrix()
# whole_dataset = normalize(whole_dataset,axis =1)
whole_dataset = normalize(whole_dataset)
train_set, test_set = train_test_split(whole_dataset, test_size = 0.2)
feed_y = train_set[:,7:8]
feed_x = train_set[:,:7]
test_x = test_set[:,:7]
test_y = test_set[:,7:8]

In [ ]:
init = tf.global_variables_initializer()
saver = tf.train.Saver([dnn.gW1,dnn.gb1,dnn.gW2,dnn.gb2])
with tf.Session() as sess:
    
    sess.run(init)
    
    batches = 100000
    
    for i in range(batches):
        
        data = next_batch(feed_x,batch_size)   
        
        label = next_batch(feed_y,batch_size)
        
        feed = {xph:data,yph:label}
        
        sess.run(train,feed)
        
        if i% 10000==0:
        
            print(sess.run(loss,feed))
    
    result = sess.run(result,feed_dict={xph:test_x})
    print('Finish!')

In [ ]:
np.sum(np.square(test_y-result))